This notebook contains the code for:
- feature engineering 
- dimensionality reduction using UMAP
- clustering using HDBSCAN

In [ ]:
import numpy as np     
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import random

In [ ]:
import umap
import hdbscan

In [ ]:
import joblib 
from joblib import Parallel, delayed

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
import sys
sys.path.append('../../utils/')

In [ ]:
from wavelets import get_wv_parameters, wv_transform

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Load metadata

In [ ]:
df_meta = pd.read_pickle('../../../data/amphioxus_metadata_final500.pickle')

In [ ]:
# create a filename column to match with the filename column in the dataset
df_meta['filename'] = df_meta['filename_video'].apply(lambda x: x.split('.avi')[0])

In [ ]:
len(df_meta.filename.unique())

# Load the data (postural and kineamtic features computed from DLC tracking)

In [ ]:
df = pd.read_hdf('../../../results/featureset_v5_08082023.h5', key='features')

# Get features from wildtype (control) animals

In [ ]:
df_merged = df.merge(df_meta, how='left', on='filename')

In [ ]:
df_control = df_merged[(df_merged['age'] > 50)&(df_merged['drugs']=='none')&(df_merged['light']=='None')]

In [ ]:
df_control.shape

In [ ]:
print(f'Number of control/ wildtype videos :{len(df_control.filename.unique())}')

# Selecting features to be used for analysis

## Speeds

In [ ]:
df_speeds = df_control.filter(like='speed')
df_speeds.shape

In [ ]:
df_speeds = df_speeds.fillna(value=-1) # Nan masking

In [ ]:
df_speeds.shape

In [ ]:
df_speeds.to_pickle('/data/temp/athira/speed_array_for_tsne_testing_control.pickle')

## Using curvatures (curvatures have been computed for dorsal points only)

In [ ]:
df_curv = df_control.filter(like='curv')
df_curv.shape

How to use curvatures in the analysis? 
- We would like to encode the temporal dynamics (how does the curvatures change along time?)
- One obvious way to do it is to use wavelets which can encode multiresolution - spatial and temporal features
- Then the question becomes, how do we use the curvatures? 
    1. Option 1: Compute wavelets for all 15 curvature points.
    2. Option 2: First use PCA to capture much of the variance in curvatures. Will that suffice? 

### Try PCA ~ akin to eigen cionas, eigen worms etc

In [ ]:
pca = PCA()
pca_curv = pca.fit_transform(df_curv)
pca_curv.shape

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('pca component')
plt.ylabel('variance explained')

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

In [ ]:
np.where(np.cumsum(pca.explained_variance_ratio_)<0.98)

if we use the first 6 pca components, it can explain > 97% of the variance in the data

In [ ]:
for i in range(6):
    df_control[f'pca_{i}'] = pca_curv[:,i]

### Compute wavelet transforms

In [ ]:
# Wavelet features has to be computed for each file

df_files = df_control.groupby(by = 'filename')

df_pca_feats = []
pca_feats = [f'pca_{i}' for i in range(6)]

for fn, group in df_files:
    df_pca_feats.append(group.filter(items = pca_feats + ['filename', 'frame']))

In [ ]:
omg0, widths = get_wv_parameters()

In [ ]:
df_wv_transforms = Parallel(n_jobs=40, verbose = 5)(delayed(wv_transform)(df, widths, omg0) 
                                                for df in df_pca_feats)

In [ ]:
df_wavelets = pd.concat(df_wv_transforms)

In [ ]:
df_wavelets.shape

# UMAP

## for speeds

In [ ]:
reducer_speeds = umap.UMAP(n_jobs=40) # assuming euclidean metrics will do fine

In [ ]:
embedding_speeds = reducer_speeds.fit_transform(df_speeds.values)
embedding_speeds.shape

In [ ]:
fn_model_speeds = f'../../../results/umap_model_speeds_17082023_all.joblib'
joblib.dump(reducer_speeds, fn_model_speeds)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(15,7))
axes.scatter(embedding_speeds[:, 0],embedding_speeds[:, 1], s=0.2)
axes.set_aspect('equal')

In [ ]:
fig, axes = plt.subplots(3,3, figsize=(15,15), sharex=True)
axes= axes.ravel()
axes[0].scatter(embedding_speeds[:, 0],embedding_speeds[:, 1], s=0.2)

hue_feats = {'mouth': df['speed_MOUTH'],
             'quirkiness': df['quirkiness'],
             'mean_dorsal_speeds': df.filter(like='speed_D').mean(axis=1),
             'mean_ventral_speeds': df.filter(like='speed_V').mean(axis=1),
             'mean_ventral_speeds': df.filter(like='speed_V').mean(axis=1),
             'mean_speeds': df.filter(like='speed_').mean(axis=1),
#              'length': df['len_sum_of_parts'],
             'mean_curv': df.filter(like='curv').abs().mean(axis=1),
             'speed_NT': df['speed_NT'],
            }

for i, key_hue in enumerate(hue_feats.keys()):
    axes[i+1].scatter(embedding_speeds[:, 0],embedding_speeds[:, 1], c= hue_feats[key_hue], s=0.2)
    axes[i+1].set_title(key_hue)
for ax in axes:  
    ax.set_aspect('equal')
fig.savefig('../../../results/umap_speeds_25082023_all.png')

## for wavelets (curvatures)

In [ ]:
df_wavelets.filter(like='wv').shape

In [ ]:
reducer_wavelets = umap.UMAP(n_jobs=40)

In [ ]:
embedding_wavelets = reducer_wavelets.fit_transform(df_wavelets.filter(like='wv').values)
embedding_wavelets.shape

In [ ]:
fn_model_wavelets = f'../../../results/umap_model_wavelets_18082023_all.joblib'
joblib.dump(reducer_wavelets, fn_model_wavelets)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(15,7))
axes.scatter(embedding_wavelets[:, 0],embedding_wavelets[:, 1], s=0.2)
axes.set_aspect('equal')

In [ ]:
fig, axes = plt.subplots(3,3, figsize=(15,15), sharex=True)
axes= axes.ravel()
axes[0].scatter(embedding_wavelets[:, 0],embedding_wavelets[:, 1], s=0.2)

hue_feats = {'mouth': df['speed_MOUTH'],
             'quirkiness': df['quirkiness'],
             'mean_dorsal_speeds': df.filter(like='speed_D').mean(axis=1),
             'mean_ventral_speeds': df.filter(like='speed_V').mean(axis=1),
             'mean_speeds': df.filter(like='speed_').mean(axis=1),
#              'length': df['len_sum_of_parts'],
             'mean_curv': df.filter(like='curv').abs().mean(axis=1),
             'speed_NT': df['speed_NT'],
            }

for i, key_hue in enumerate(hue_feats.keys()):
    axes[i+1].scatter(embedding_wavelets[:, 0],embedding_wavelets[:, 1], c= hue_feats[key_hue], s=0.2)
    axes[i+1].set_title(key_hue)
for ax in axes:  
    ax.set_aspect('equal')
# fig.savefig('../../../results/umap_wavelets_18082023_all.png')

## try tsne for wavelets

In [ ]:
df_wavelets.filter(like='wv').values.shape

In [ ]:
df_wavelets_filt = df_wavelets.filter(like='wv').dropna()
df_wavelets_filt.shape

In [ ]:
df_wavelets_filt.to_pickle('/data/temp/athira/wavelet_array_for_tsne_testing_control.pickle')

# Load UMAP models and compute the embeddings

In [ ]:
fn_model_speeds = f'../../../results/umap_model_speeds_17082023_control.joblib'
fn_model_wavelets = f'../../../results/umap_model_wavelets_17082023_control.joblib'

In [ ]:
umap_speeds =  joblib.load(fn_model_speeds)
umap_wavelets =  joblib.load(fn_model_wavelets)

In [ ]:
embedding_speeds = umap_speeds.transform(df_speeds.values)
embedding_speeds.shape

In [ ]:
embedding_wavelets = umap_wavelets.transform(df_wavelets.filter(like='wv').values)
embedding_wavelets.shape

# Clustering 

In [ ]:
c_pal = sns.color_palette('tab10', 10)

In [ ]:
embedding_4d = np.hstack([embedding_speeds, embedding_wavelets])
embedding_4d.shape

##  scaling of inputs

In [ ]:
embedding_4d_scaled = MinMaxScaler().fit_transform(embedding_4d)

In [ ]:
for i in range(embedding_4d_scaled.shape[1]):
    feat_max  = np.max(embedding_4d_scaled[:,i])
    print(feat_max)

## 5D- add speed MOUTH variable separately

In [ ]:
embedding_5d = np.hstack([embedding_4d, df_speeds['speed_MOUTH'].values.reshape(-1,1)])
embedding_5d.shape

## HDBSCAN

In [ ]:
clusterer_scaled = hdbscan.HDBSCAN(
    min_samples= 1, #larger values implies more points considered as noise
    min_cluster_size= 10000, #smallest size grouping to be considered as a cluster
    cluster_selection_epsilon= 1,
    cluster_selection_method='leaf',
    prediction_data=True, 
    )

In [ ]:
labels_scaled = clusterer_scaled.fit_predict(embedding_4d)

In [ ]:
filename_hdbscan = f'../../../results/hdbscan_4Dumap_model_26082023b_control.joblib'
joblib.dump(clusterer_scaled, filename_hdbscan)

In [ ]:
dict_clusters_scaled = {f'cluster_{i}':np.sum(labels_scaled==i) for i in list(np.unique(labels_scaled))}
dict_clusters_scaled

In [ ]:
dict_clusters_scaled = {f'cluster_{i}':np.sum(labels_scaled==i) for i in list(np.unique(labels_scaled))}
dict_clusters_scaled

In [ ]:
c_pal = sns.color_palette('tab20', 20)

In [ ]:
c_dict_scaled = {i: c_pal[i+1] for i in np.unique(labels_scaled)}
labels_c_scaled = [c_dict_scaled[lab] for lab in labels_scaled]

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(15,7))
axes= axes.ravel()
axes[0].scatter(
    embedding_speeds[:, 0],
    embedding_speeds[:, 1], c=labels_c_scaled, s=0.01)
axes[1].scatter(
    embedding_wavelets[:, 0],
    embedding_wavelets[:, 1], c=labels_c_scaled, s=0.01)

markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in c_dict_scaled.values()]
plt.legend(markers, c_dict_scaled.keys(), numpoints=1)

for ax in axes:
    ax.set_aspect('equal')
    
# fig.savefig('../../../results/cluster_umap4dscaled_25082023a_control.png')

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(15,7))
axes= axes.ravel()
axes[0].hist2d(
    embedding_speeds[:, 0],
    embedding_speeds[:, 1], bins=(150,150), density=True)
axes[1].hist2d(
    embedding_wavelets[:, 0],
    embedding_wavelets[:, 1], bins=(150,150), density=True)


for ax in axes:
    ax.set_aspect('equal')

# sampled subset

## HDBSCAN on sampled dataset

In [ ]:
import random

In [ ]:
embedding_4d.shape

In [ ]:
sampled_4d  = random.sample(list(embedding_4d), 500000)

In [ ]:
sampled_4d = np.reshape(np.concatenate(sampled_4d),(-1,4))

In [ ]:
plt.scatter(sampled_4d[:,0], sampled_4d[:,1], s=0.01)

In [ ]:
clusterer_scaled = hdbscan.HDBSCAN(
    min_samples= 1, #larger values implies more points considered as noise
    min_cluster_size= 200, #smallest size grouping to be considered as a cluster
    cluster_selection_epsilon= 1,
    cluster_selection_method='leaf',
    prediction_data=True, 
    )

In [ ]:
labels_sampled = clusterer_scaled.fit_predict(sampled_4d)

In [ ]:
dict_clusters_sampled = {f'cluster_{i}':np.sum(labels_sampled==i) for i in list(np.unique(labels_sampled))}
dict_clusters_sampled

## DBSCAN on sampled dataset

# GMM

In [ ]:
from sklearn.mixture import BayesianGaussianMixture

In [ ]:
bgm = BayesianGaussianMixture(n_components=10, random_state=42, covariance_type='diag', 
                              weight_concentration_prior_type="dirichlet_process",
                              weight_concentration_prior=100000,
                              max_iter=1500
                             )

In [ ]:
bgm.fit(sampled_4d)

In [ ]:
bgm.means_

In [ ]:
clusters_gmm = bgm.predict(sampled_4d)

In [ ]:
len(clusters_gmm)

In [ ]:
np.unique(clusters_gmm)

In [ ]:
c_pal1 = sns.color_palette('tab10', 10)
clusters_c_gmm = [c_pal1[lab] for lab in clusters_gmm]

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(15,7))
axes= axes.ravel()
axes[0].scatter(
    sampled_4d[:, 0],
    sampled_4d[:, 1], c=clusters_c_gmm, s=0.01)
axes[1].scatter(
    sampled_4d[:, 2],
    sampled_4d[:, 3], c=clusters_c_gmm, s=0.01)

## HDBSCAN - 5D

In [ ]:
embedding_5d = np.hstack([embedding_4d, df_speeds['speed_MOUTH'].values.reshape(-1,1)])
embedding_5d.shape

In [ ]:
embedding_5d_scaled = MinMaxScaler().fit_transform(embedding_5d)

In [ ]:
clusterer_5d = hdbscan.HDBSCAN(
    min_samples= 1, #larger values implies more points considered as noise
    min_cluster_size= 10000, #smallest size grouping to be considered as a cluster
    cluster_selection_epsilon=0.01,
    cluster_selection_method='leaf',
    prediction_data=True, 
    )

In [ ]:
embedding_5d_scaled

In [ ]:
labels_scaled_5d = clusterer_5d.fit_predict(embedding_5d_scaled)

In [ ]:
dict_clusters_scaled_5d = {f'cluster_{i}':np.sum(labels_scaled_5d==i) for i in list(np.unique(labels_scaled_5d))}
dict_clusters_scaled_5d